## Library

In [1]:
! pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-57rzk8vu
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-57rzk8vu
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
from transformers import pipeline
from normalizer import normalize
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Test Data 

In [3]:
test_df = pd.read_csv("/kaggle/input/nlp-competition-cuet-ete-day-2023/test.csv")
test_df

,Id,summary
0,0,আমার কোন উপন্যাস তখনি ভাল লাগে যখন আমি এর চরিত...
1,1,বইঃ দ্য সান অফ লাইট (রামেসিস #1) লেখকঃ ক্রিশ্চ...
2,2,বাংলাদেশ আমার জন্মভূমি। সাতপুরুষের ভিটে-মাটি। ...
3,3,বিংশ শতাব্দীর বাঙালি জনপ্রিয় কথাসাহিত্যিকদের ...
4,4,অনেক সুন্দর এবং অসাধারণ একটি বই.... মধ্যবিত্ত...
...,...,...
682,682,বইটির কাহিনী আবর্তিত হয়েছে যুক্তরাষ্ট্রের দুই ...
683,683,"ফরাসি বিপ্লব মানে বুর্জোয়া বিপ্লব, আদি সমাজতা..."
684,684,শুভ্র চরিত্রটি আমরা প্রথম দেখতে পাই ‘দারুচিনি...
685,685,রকমারি_রিভিউ_প্রতিযোগিতা_জুন\r\n\r\nপালোমিনো ম...


In [4]:
filtered_test = test_df[lambda x: x["summary"].str.contains("[A-Za-z0-9]")]
filtered_test

,Id,summary
1,1,বইঃ দ্য সান অফ লাইট (রামেসিস #1) লেখকঃ ক্রিশ্চ...
5,5,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা বই:আর্কন লেখক...
6,6,পক্ষপাতিপূর্ণ বই। লেখক অনেক কষ্টকরে জিন্নাহক...
8,8,বই- দ্য বেষ্ট লেইড প্ল্যানস।\r\nলেখক- সিডনী শে...
10,10,very interesting
...,...,...
671,671,হিমু সমগ্র-১ নিয়েছি...দারুন ছিল বইটি...\r\nকিন...
673,673,#রহস্যময়_বিজ্ঞান_জগত ও #রকমারি_ডট_কম #বুক_রিভ...
676,676,"বইটিতে ইমরানের ক্রিকেট ক্যারিয়ার, ইতিহাস, বিশ্..."
680,680,মোসাদকে বলা হয় বিশ্বের সবচেয়ে ভয়াবহ ও দুর্ধর্ষ...


In [5]:
bangla_alnum = '[০১২৩৪৫৬৭৮৯অআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহড়ঢ়য়ৎংঃঁ]'

filtered_test_2 = filtered_test[lambda x: x["summary"].str.contains(bangla_alnum)]

In [6]:
filtered_test_2

,Id,summary
1,1,বইঃ দ্য সান অফ লাইট (রামেসিস #1) লেখকঃ ক্রিশ্চ...
5,5,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা বই:আর্কন লেখক...
6,6,পক্ষপাতিপূর্ণ বই। লেখক অনেক কষ্টকরে জিন্নাহক...
8,8,বই- দ্য বেষ্ট লেইড প্ল্যানস।\r\nলেখক- সিডনী শে...
14,14,সায়রা সায়েন্টিস্ট লেখকঃ মুহম্মদ জাফর ইকবাল মূল...
...,...,...
671,671,হিমু সমগ্র-১ নিয়েছি...দারুন ছিল বইটি...\r\nকিন...
673,673,#রহস্যময়_বিজ্ঞান_জগত ও #রকমারি_ডট_কম #বুক_রিভ...
676,676,"বইটিতে ইমরানের ক্রিকেট ক্যারিয়ার, ইতিহাস, বিশ্..."
680,680,মোসাদকে বলা হয় বিশ্বের সবচেয়ে ভয়াবহ ও দুর্ধর্ষ...


In [7]:
# texts with only english alphabets
test_eng = filtered_test.loc[filtered_test.index.difference(filtered_test_2.index), ]

In [8]:
# texts without only english alphabets
test_not_eng = test_df.loc[test_df.index.difference(test_eng.index), ]

In [9]:
test_not_eng

,Id,summary
0,0,আমার কোন উপন্যাস তখনি ভাল লাগে যখন আমি এর চরিত...
1,1,বইঃ দ্য সান অফ লাইট (রামেসিস #1) লেখকঃ ক্রিশ্চ...
2,2,বাংলাদেশ আমার জন্মভূমি। সাতপুরুষের ভিটে-মাটি। ...
3,3,বিংশ শতাব্দীর বাঙালি জনপ্রিয় কথাসাহিত্যিকদের ...
4,4,অনেক সুন্দর এবং অসাধারণ একটি বই.... মধ্যবিত্ত...
...,...,...
682,682,বইটির কাহিনী আবর্তিত হয়েছে যুক্তরাষ্ট্রের দুই ...
683,683,"ফরাসি বিপ্লব মানে বুর্জোয়া বিপ্লব, আদি সমাজতা..."
684,684,শুভ্র চরিত্রটি আমরা প্রথম দেখতে পাই ‘দারুচিনি...
685,685,রকমারি_রিভিউ_প্রতিযোগিতা_জুন\r\n\r\nপালোমিনো ম...


In [10]:
# Remove English alphanumeric values
alpha_pat = "[a-zA-z0-9]"

test_not_eng["summary"] = test_not_eng["summary"].str.replace(alpha_pat, "", regex=True)

In [11]:
test_not_eng

,Id,summary
0,0,আমার কোন উপন্যাস তখনি ভাল লাগে যখন আমি এর চরিত...
1,1,বইঃ দ্য সান অফ লাইট (রামেসিস #) লেখকঃ ক্রিশ্চি...
2,2,বাংলাদেশ আমার জন্মভূমি। সাতপুরুষের ভিটে-মাটি। ...
3,3,বিংশ শতাব্দীর বাঙালি জনপ্রিয় কথাসাহিত্যিকদের ...
4,4,অনেক সুন্দর এবং অসাধারণ একটি বই.... মধ্যবিত্ত...
...,...,...
682,682,বইটির কাহিনী আবর্তিত হয়েছে যুক্তরাষ্ট্রের দুই ...
683,683,"ফরাসি বিপ্লব মানে বুর্জোয়া বিপ্লব, আদি সমাজতা..."
684,684,শুভ্র চরিত্রটি আমরা প্রথম দেখতে পাই ‘দারুচিনি...
685,685,রকমারিরিভিউপ্রতিযোগিতাজুন\r\n\r\nপালোমিনো মোলে...


In [12]:
test_not_eng['summary'] = pd.DataFrame(test_not_eng['summary'].map(lambda x: normalize(x)))

In [13]:
test_not_eng

,Id,summary
0,0,আমার কোন উপন্যাস তখনি ভাল লাগে যখন আমি এর চরিত...
1,1,বইঃ দ্য সান অফ লাইট (রামেসিস #) লেখকঃ ক্রিশ্চি...
2,2,বাংলাদেশ আমার জন্মভূমি। সাতপুরুষের ভিটে-মাটি। ...
3,3,বিংশ শতাব্দীর বাঙালি জনপ্রিয় কথাসাহিত্যিকদের ...
4,4,অনেক সুন্দর এবং অসাধারণ একটি বই.... মধ্যবিত্ত ...
...,...,...
682,682,বইটির কাহিনী আবর্তিত হয়েছে যুক্তরাষ্ট্রের দুই...
683,683,"ফরাসি বিপ্লব মানে বুর্জোয়া বিপ্লব, আদি সমাজতা..."
684,684,শুভ্র চরিত্রটি আমরা প্রথম দেখতে পাই 'দারুচিনি ...
685,685,রকমারিরিভিউপ্রতিযোগিতাজুন পালোমিনো মোলেরোকে খু...


In [14]:
test_eng

,Id,summary
10,10,very interesting
11,11,"ai uponnas ta ami porechi, humayun sir jothart..."
45,45,amr life e pora shera boi gular moddhe ekta
80,80,Vlo boi..
88,88,very interesting book
99,99,A must read book if you want to know the histo...
108,108,Good Try
111,111,good
142,142,Just usual Himu. Nothing sort of extra-ordinar...
149,149,bogus


## Load Finetuned Model

### Model 1(Dealing with Bangla texts)

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


model_directory1 = '/kaggle/input/baseline-banglabert/banglabert/sequence_classification/outputs/checkpoint-870'
    
tokenizer1 = AutoTokenizer.from_pretrained(model_directory1)
model1 = AutoModelForSequenceClassification.from_pretrained(model_directory1)


### Model2(Dealing with English texts)

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


model_directory2 = '/kaggle/input/baseline-banglishbert/banglabert/sequence_classification/outputs/checkpoint-264'
    
tokenizer2 = AutoTokenizer.from_pretrained(model_directory2)
model2 = AutoModelForSequenceClassification.from_pretrained(model_directory2)

In [17]:
def preprocess_function1(examples):
    return tokenizer1(examples['summary'], max_length=512, padding=True, truncation=True)
def preprocess_function2(examples):
    return tokenizer2(examples['summary'], max_length=512, padding=True, truncation=True)

## test_not_eng

In [18]:
test_not_eng
test_not_eng_ds = Dataset.from_pandas(test_not_eng)
test_not_eng_ds_enc = test_not_eng_ds.map(preprocess_function1, batched=True)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
trainer1 = Trainer(
    model=model1,
    tokenizer=tokenizer1
)

In [20]:
test_preds1 = trainer1.predict(test_not_eng_ds_enc)
logits1 = test_preds1.predictions
probs1 = np.exp(logits1) / np.sum(np.exp(logits1), axis=-1, keepdims=True)

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [21]:
predictions1 = np.argmax(probs1, axis=-1)

In [22]:
sub1 = pd.DataFrame()
sub1['Id'] = test_not_eng['Id']
sub1['Label'] = predictions1
sub1

,Id,Label
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
682,682,1
683,683,3
684,684,0
685,685,1


### test_eng

In [23]:
test_eng
test_eng_ds = Dataset.from_pandas(test_eng)
test_eng_ds_enc = test_eng_ds.map(preprocess_function2, batched=True)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
trainer2 = Trainer(
    model=model2,
    tokenizer=tokenizer2
)

In [25]:
test_preds2 = trainer2.predict(test_eng_ds_enc)
logits2 = test_preds2.predictions
probs2 = np.exp(logits2) / np.sum(np.exp(logits2), axis=-1, keepdims=True)

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [26]:
predictions2 = np.argmax(probs2, axis=-1)

In [27]:
sub2 = pd.DataFrame()
sub2['Id'] = test_eng['Id']
sub2['Label'] = predictions2
sub2

,Id,Label
10,10,3
11,11,0
45,45,0
80,80,3
88,88,3
99,99,3
108,108,0
111,111,0
142,142,0
149,149,3


In [28]:
len(sub2)

30

In [29]:
sub=pd.concat([sub1,sub2])
sub=sub.sort_index()
sub

,Id,Label
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
682,682,1
683,683,3
684,684,0
685,685,1


In [30]:
sub.to_csv('submission.csv', index=False)
sub.head()

,Id,Label
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
